# bringing together all the samples for this run
+ MutID refers to single mutation in any samples
+ SampleMutID refers to a mutation in a specific sample

### settings

In [ ]:
import os
home = "/Users/martinscience"
home = "/Users/mahtin"
folder = os.path.join(home, "Dropbox/Icke/Work/somVar/NHL-DLBCL/NextNEBNext")

## get AG Krönke info
+ Patient - time point information is combined in SampleID
+ these samples have to be allocated to the full subpopulation PCRs without overlaps
+ load PCR setup --> load_kroenke_PCR()
    * sample :: Primer=MutID
+ load the primer coords and mutation coords for each MutID --> load_kroenke_primer
    * MutID --> MutStartEnd + AmpliconRange
    * we need primer coords for assessing overlaps

In [ ]:
def num2well(num):
    '''
    loads PCR setup for association sample :: MutID(PrimerName) -> SampleMutID
    '''
    
    string = "ABCDEFGH"
    char = string[(num - 1) % 8]
    count = int((num - 1) / 8) + 1
    return f"{char}{count}"


def load_kroenke_SU(coop_file):
    '''
    loads SampleID :: UPN
    '''
    return pd.read_excel(coop_file, sheet_name="Pat_Proben", engine="openpyxl").loc[:, ['SampleID', 'UPN']].query('SampleID != "na"')

def load_kroenke_PCR(coop_file):
    '''
    takes care of everything
    '''
    # load into df the 
    
    coop_df = pd.read_excel(coop_file, sheet_name="PCR", engine="openpyxl")
    # edit the columns
    # list(coop_df.columns)
    # Primername does not directly translate into MutID (is unique only for each patient!)
    # strip() is important because of trailing whitespace!!!
    coop_df['MutID'] = coop_df['PrimerName'].str.strip()
    # SampleMutID is is fully unique without the UPN
    coop_df['SampleMutID'] = coop_df['SampleID'] + ":" + coop_df['MutID']
    
    # however, Primer+UPN is unique --> MutID
    # first, get SampleID::UPN association
    SU_df = load_kroenke_SU(coop_file)
    coop_df = coop_df.merge(SU_df)
    coop_df['MutID'] = coop_df['MutID'] + "-UPN" + coop_df['UPN'].astype(str)
    
    # compute the well coordinates from the PCR-Belegung
    coop_df['Well'] = coop_df['PCR-Belegung'].apply(num2well)
    # make Well categorical for easy sorting
    coop_df['Well'] = pd.Categorical(coop_df['Well'], [a + str(b +1) for b in range(8) for a in "ABCDEFGH"])
    # insert the info about repeated PCR
    coop_df['status'] = np.char.multiply("PCRrepeat", (coop_df['PCR-fail'] == coop_df['PCR-fail']).astype(int))
    # reduce to important columns
    coop_df = coop_df.loc[:, [
        'SampleMutID',
        'MutID',
        'Well',
        'SampleID',
        'status'
    ]]
    
    return coop_df

def load_kroenke_primer(coop_file):
    '''
    load the primer coords and the MutID-specific Mutations
    MutID --> MutStartEnd + AmpliconRange
    '''
    
    # load the file
    coop_primer_df = pd.read_excel(coop_file, sheet_name="Mutationen_hg38", engine="openpyxl")
    
    
    # chr_pat = r"(chr[0-9XY]+):([0-9]+)-([0-9]+)"

    # get the MutID (PrimerName + UPN) from Primer_Name_F + UPN
    coop_primer_df['MutID'] = coop_primer_df['Primer_Name_F '].str.replace(r"_F( and.*$)?", "") + "-UPN" + coop_primer_df['UPN'].astype(str)
    
    # make Chr categorical for better sorting
    coop_primer_df['Chr'] = pd.Categorical(coop_primer_df['Chr'], [f"chr{i}" for i in range(23)] + ['chrX', 'chrY'])
    coop_primer_df = coop_primer_df.loc[:, [
         'MutID',
         'Gene',
         'Chr',
         'Mut Start ',
         'Mut End ',
         'fwdPrimer',
         'revPrimer',
         'Temp',
         'AmpliconRange',
    ]].rename({'Mut Start ': "Start", 'Mut End ': "End"}, axis=1)
    return coop_primer_df

def extract_AmpRange(df):
    df[["AmpStart", "AmpEnd"]] = df['AmpliconRange'].str.extract(r"chr[0-9XY]+:([0-9]+)-([0-9]+)")
    for col in ["AmpStart", "AmpEnd"]:
        df[col] = df[col].astype(int)
    return df.drop("AmpliconRange", axis=1)

In [ ]:
coop_file = os.path.join(folder, "Final_list_CVK.xlsx")
PCR_df = load_kroenke_PCR(coop_file)
primer_df = load_kroenke_primer(coop_file)
coop_df = PCR_df.merge(primer_df, on="MutID").sort_values("Well")
coop_df = extract_AmpRange(coop_df)
coop_df['Project'] = "AG_Kroenke"
coop_df[:10]

### WARNING!!!
+ coop_df contains identicat SampleMutIDs with different mutations
+ that is G5, B6, E6 and H6!!

In [ ]:
coop_df.sort_values('SampleMutID').query('Gene == "TP53"').loc[coop_df['MutID'].str.startswith("TP53_e7_1"),:]

## get our mutation list

In [ ]:
def get_pop_df(pop_file):

    # cycle through the sheets and append
    pop_dfs = []
    for sheet in [f"NHL{i}" for i in [1,2,8]]:
        pop_df = pd.read_excel(pop_file, sheet_name=sheet, engine="openpyxl")
        # reduce to the relevant columns plus MutID
        pop_df = pop_df.loc[:, [col for col in pop_df.columns if col != "LEGENDE" and not "Unnamed" in col]]
        # add the G for all working samples in Vanessas columns
        pop_df = pop_df.fillna("G")
        pop_df['SampleID'] = sheet
        pop_df['MutID'] = pop_df['MutID'].str.replace("_", "-")
        for c in pop_df.columns:
            if c != "Primer":
                pop_df[c] = pop_df[c].str.replace(r" .*", "")
        pop_dfs.append(pop_df)
    pop_df = pd.concat(pop_dfs).rename(dict(Primer="Well"), axis=1)
    
    # compute the well coordinates
    pop_df['Well'] = pop_df['Well'].apply(num2well)
    # make Well categorical for easy sorting
    pop_df['Well'] = pd.Categorical(pop_df['Well'], [a + str(b +1) for b in range(8) for a in "ABCDEFGH"])
    return pop_df

def get_mut_df(mut_file):
    # get the output columns that should be the same for our lists
    select_cols = ['MutID',
     'SampleID',
     'Gene',
     'Chr',
     'Start',
     'End',
     'fwdPrimer',
     'revPrimer',
     'Temp',
     'AmpliconRange']

    # cycle through the sheets and append
    mut_dfs = []
    for sheet in [f"NHL{i}" for i in [1,2,8]]:
        mut_df = pd.read_excel(mut_file, sheet_name=sheet, engine="openpyxl")
        # get SampleID from sheet name
        mut_df['SampleID'] = sheet
        # get MutID from Chr and Start
        mut_df['MutID'] = mut_df['Chr'] + "-" + mut_df['Start'].astype(int).astype(str)
        mut_df['MutID'] = mut_df['Chr'] + "-" + mut_df['Start'].astype(int).astype(str)
        # rename columns for harmonizing with COOP
        mut_df = mut_df.rename({"Note": "Temp", 'rev_Primer': 'revPrimer', 'fwd-Primer':'fwdPrimer'}, axis=1).loc[:, select_cols]
        mut_dfs.append(mut_df)
    mut_df = pd.concat(mut_dfs)
    return mut_df


def make_indices(df):
    '''
    assigns indices from SampleID and returns the df with indices
    '''
    df2 = df.copy()
    index_df = df2.groupby("SampleID").count().loc[:,"SampleMutID"].reset_index().reset_index().drop("SampleMutID", axis=1)
    return df2.merge(index_df, on="SampleID")


def merge_df(pop_df, mut_df):
    '''
    reshape the pop_df and add info fields
    '''
    
    select_cols = ['MutID',
     'SampleID',
     'Gene',
     'Chr',
     'Start',
     'End',
     'fwdPrimer',
     'revPrimer',
     'Temp',
     'AmpliconRange']
    
    
    nhl_df = pop_df.merge(mut_df, on=["MutID", "SampleID"]).set_index(select_cols + ['Well']).stack().reset_index().rename({0:"status", "level_11":"type"}, axis=1)
    nhl_df['Project'] = "NHL"
    nhl_df['SampleID'] = nhl_df['SampleID'] + "-" + nhl_df['type']
    nhl_df['SampleMutID'] = nhl_df['SampleID'] + ":" + nhl_df['MutID']
    # make Chr categorical
    nhl_df['Chr'] = pd.Categorical(nhl_df['Chr'], [f"chr{i}" for i in range(23)] + ['chrX', 'chrY'])
    # extract Amplicon Range
    nhl_df = extract_AmpRange(nhl_df)
    # make primary index by counting the SampleIDs
    nhl_df = make_indices(nhl_df)
    return nhl_df.sort_values(['SampleID', 'Chr', 'Start'], ascending=True)


def get_index_df(org_df):
    '''
    returns the sum of sequenczing samples per index
    '''
    
    # assign microliters
    df = org_df.copy()
    df['µl'] = 2
    df.loc[df['status'].str.startswith("F"), 'µl'] = 0
    df.loc[df['status'].str.startswith("P"), 'µl'] = 4
    index_df = df.loc[~df['status'].str.startswith("F")].groupby("index").agg({"MutID":"count", "Project":"first", "µl":"sum", "SampleID":"first"}).rename(dict(MutID="sampleCount"), axis=1).reset_index()
    
    return index_df

In [ ]:
pop_file = os.path.join(folder, "Subpop_PCR2.xlsx")
pop_df = get_pop_df(pop_file)
### merge the Primers with respective mutations
mut_file = os.path.join(folder, "Subpop_mutations.xlsx")
mut_df = get_mut_df(mut_file)
nhl_df = merge_df(pop_df, mut_df).sort_values(['SampleID', 'Chr', 'Start'], ascending=True)
nhl_df

## merging coop into nhl_df

In [ ]:
def showOverlap(df, padding=10):
    '''
    finds overlaps between mutations
    overlaps are defined as mutations between different samples at overlapping positions in the same index
    
    requires that identical SampleMutID (same sample, same Primers for different mutation) has been deduplicated
    --> has to be improved by gap group assignment (LEGACY code)!
    '''
    
    # checks if the df has any overlaps as described above
    full_df = df.copy()
    # df has to be sorted by index, amplicon start site and then by SampleID
    full_df = full_df.sort_values(['index', "Chr", "AmpStart", "SampleID"])
    # make an integer for any kind of overlap
    full_df['internal_overlap'] = ((full_df['AmpStart'] + padding < full_df.shift(1)['AmpEnd']) & (full_df['Chr'] == full_df.shift(1)['Chr']) & (full_df['index'] == full_df.shift(1)['index']) & (full_df['SampleID'] != full_df.shift(1)['SampleID'])).astype(int)
    return full_df


def hasOverlap(df, padding=10):
    # boolean checks if the df has any overlaps as described above
    full_df = showOverlap(df)
    return len(full_df.query("internal_overlap == 1")) > 0



# get the index with the fewest sample for equal allocation
def get_min_index(df, black_list=[], offset=0):
    '''
    get the index containing fewest samples from samples exluding black_list
    '''
    # remove -1 index
    index_df = get_index_df(df.query("index != -1")).query("SampleID not in @black_list").sort_values("sampleCount", ascending=True).iloc[offset]['index']
    
    return index_df

# black_list = [f"NHL{i}-{t}" for i in [1,2,8] for t in ["WB", "GR"]]
# index_df = get_index_df(nhl_df)
# index_df.sort_values("sampleCount")[:10]
# get_min_index(nhl_df, offset=1, black_list=black_list)


def merge_indices(main_df, df, black_list=[]):
    '''
    takes an indexed main_df and tries to merge df into it
    1) non-overlapping rows in df are bundled into one index
    2) internally overlapping mutations are assigned to index in main_df with fewest samples
    '''
    # remove any duplicate SampleMutID
    # they do exist!! need to be seen by pipeline but not by indexing
    df = df.drop_duplicates("SampleMutID")

    # set arbitrary index for checking overlaps
    df['index'] = -1
    # assign internal overlaps
    df = showOverlap(df)
    
    #### ASSIGN NON-OVERLAPPING MUTATIONS TO NEW INDEX ########
    
    
    # make new index for non_overlapping samples
    next_index = get_index_df(main_df)['index'].max() + 1
    print("Assigning non-overlapping bulk to index = ", next_index)
    df.loc[df["internal_overlap"] == 0, "index"] = next_index
    
    #### ASSIGN OVERLAPPING MUTATIONS ########
    # add overlap_df to main_df and 
    all_df = pd.concat([main_df, df.query('internal_overlap == 1')]).reset_index(drop=True).drop('internal_overlap', axis=1)

    overlap_df = all_df.query('index == -1')
    print(f"Assigning overlapping {len(overlap_df.index)} samples to existing index groups")
    
    # step through rows
    for i, row in overlap_df.iterrows():
        # get the current index
        index = row.name
        new_index = get_min_index(all_df, black_list=black_list)
        print(f"Assigning SampleMutID {row['SampleMutID']} (row {index})")
        all_df.loc[index,'index'] = new_index
        offset = 1
        while hasOverlap(all_df.query("index > -1")):
            print(f"SampleMutID {row['SampleMutID']} hasOverlap in index {new_index}!")
            # move to next index in line using offset
            new_index = get_min_index(all_df, offset=offset, black_list=black_list)
            all_df.loc[index,'index'] = new_index
            offset += 1
        print(f"SampleMutID {row['SampleMutID']} has been assigned to index {new_index}!")
    
    # bringing in the non_overlapping bulk mutations
    total_df = pd.concat([all_df, df.query('index > -1').drop('internal_overlap', axis=1)]).reset_index(drop=True).sort_values(["index", "Well"])
    return total_df
    

def add_info(all_df):
    '''
    custom function to bring certain data from the main df into the index pool output
    add stuff to the info field which will be used in the pool output
    '''
    all_df['info'] = all_df['SampleID'] + " (" + all_df['Well'].astype(str) + ")"
    all_df['index'] = all_df['index'].astype(int)
    # if no status, then status = "G"
    all_df.loc[all_df['status'] == "", "status"] = "G"
    # add info for non "G"
    all_df.loc[all_df['status'] != "G", 'info'] = all_df['info'] + " (" + all_df['status'] + ")"
    # add epic fail for "F"
    all_df.loc[all_df['status'].str.startswith("F"), 'info'] = "EPIC FAIL------" + all_df['info'] + "---------EPIC FAIL"
    all_df = all_df.sort_values(['index', "SampleID", "Well"])
    return all_df


def make_index_table(org_df):
    '''
    creates a table with a column per index
    '''
    
    df = add_info(org_df)
    # get the maximum rows for initiating a clean df
    max_rows = get_index_df(df)['sampleCount'].max()

    table_df = pd.DataFrame(index=range(max_rows))
    for i in df['index'].unique():
        table_df[f"index{i}"] = df.query('index == @i').reset_index()['info']
    return table_df


def save2excel(total_df, out_excel):
    
    with pd.ExcelWriter(out_excel, engine="openpyxl") as writer:
        total_df.to_excel(writer, sheet_name="all", index=False)
        index_df = get_index_df(total_df)
        index_df.to_excel(writer, sheet_name="index", index=False)
        table_df = make_index_table(total_df)
        table_df.to_excel(writer, sheet_name="pools", index=False)
    return index_df

In [ ]:
black_list = [f"NHL{i}-{t}" for i in [1,2,8] for t in ["WB", "GR"]]

total_df = merge_indices(nhl_df, coop_df, black_list=black_list)


In [ ]:
# get the index with the fewest sample for equal allocation
def get_min_index(df, black_list=[], white_list=[], offset=0):
    '''
    get the index containing fewest samples from samples exluding black_list
    '''
    # remove -1 index
    df = df.query("index != -1")
    
    # exclude samples from black_list
    df = df.query("SampleID not in @black_list").query("index not in @black_list")
    if len(white_list):
        df = df.query("SampleID in @white_list or index in @white_list ")
    index_df = get_index_df(df).sort_values("sampleCount", ascending=True).iloc[offset]['index']
    
    return index_df

black_list = [f"NHL{i}-{t}" for i in [1,2,8] for t in ["WB", "GR"]]
index_df = get_index_df(nhl_df)
index_df.sort_values("sampleCount")[:10]
get_min_index(nhl_df, offset=1, black_list=black_list, white_list=[4])

In [ ]:
    offset = 0
    black_list = [f"NHL{i}-{t}" for i in [1,2,8] for t in ["WB", "GR"]]
    black_list = []
    print(black_list)
    white_list = [1,2]
    df = nhl_df.copy()
    # remove -1 index
    df = df.query("index != -1")
    
    # exclude samples from black_list
    df = df.query("SampleID not in @black_list").query("index not in @black_list")
    if len(white_list):
        print("Using white liste")
        df = df.query("SampleID in @white_list or index in @white_list ")
        
    df
    index_df = get_index_df(df).sort_values("sampleCount", ascending=True).iloc[offset]['index']

In [ ]:
total_df.query('Project == "AG_Kroenke"')
get_index_df(total_df)

# add_info(total_df)
# table_df = make_index_table(total_df)
# table_df

+ the NHL1-pools (index 0 to 9 all have just 36 samples each)
+ cycle through these indizes and divide the coop-samples to these


### save to Excel

In [ ]:
out_excel = os.path.join(folder, "NebNextSetupTest.xlsx")
save2excel(total_df, out_excel)